In [1]:
from pybaseball import statcast_batter
from pybaseball import playerid_lookup
import pandas as pd
from scipy.stats import zscore
from pybaseball import cache
from pyspark.sql import SparkSession

cache.enable()


In [2]:
# # Sample list of player names
# player_names = ['david ortiz', 'mike trout', 'bryce harper']  # Add more player names as needed

# # Initialize an empty DataFrame to store the OPS data
# ops_data = pd.DataFrame(columns=['Player', 'Year', 'OPS'])

# # Loop through each player name
# for player_name in player_names:
#     # Lookup player ID using playerid_lookup
#     player_info = playerid_lookup(player_name.split()[1], player_name.split()[0])
    
#     if not player_info.empty:
#         player_id = player_info['key_mlbam'].iloc[0]
        
#         # Fetch player data for the desired date range
#         player_data = statcast_batter(start_dt='2008-04-01', end_dt='2023-08-13', player_id=player_id)
        
#         player_data['game_date'] = pd.to_datetime(player_data['game_date'])
#         # Calculate OPS for each year
#         yearly_ops = player_data.groupby(player_data['game_date'].dt.year)['woba_value'].mean()
        
#         # Append OPS data to the DataFrame
#         ops_data = ops_data.append(pd.DataFrame({'Player': [player_name] * len(yearly_ops),
#                                                  'Year': yearly_ops.index,
#                                                  'OPS': yearly_ops}), ignore_index=True)
#     else:
#         print(f"Player {player_name} not found.")

# # Display the collected OPS data
# ops_data

In [3]:
from pybaseball import statcast_pitcher_expected_stats

# Get data for all pitchers with a minimum of 150 plate appearances against in  2019, 2021, 2022, 2023
pitcher_data_2019 = statcast_pitcher_expected_stats(2019, 150)
pitcher_data_2021 = statcast_pitcher_expected_stats(2021, 150)
pitcher_data_2022 = statcast_pitcher_expected_stats(2022, 150)
pitcher_data_2023 = statcast_pitcher_expected_stats(2023, 150)

# Concatenate the DataFrames
pitcher_combined_data = pd.concat([pitcher_data_2019, pitcher_data_2021, pitcher_data_2022, pitcher_data_2023], ignore_index=True)

# Display the column names of the combined DataFrame
pitcher_combined_data

,last_name,first_name,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,xera,era_minus_xera_diff
0,Bauer,Trevor,545333,2019,911,557,0.230,0.223,0.007,0.429,0.402,0.027,0.316,0.310,0.006,4.48,4.23,0.249
1,Lynn,Lance,458681,2019,875,562,0.243,0.222,0.021,0.390,0.372,0.018,0.294,0.283,0.011,3.67,3.48,0.192
2,Gonzales,Marco,594835,2019,866,657,0.264,0.254,0.010,0.422,0.407,0.015,0.311,0.311,0.000,3.99,4.26,-0.270
3,Minor,Mike,501985,2019,863,588,0.244,0.228,0.016,0.395,0.389,0.006,0.301,0.298,0.003,3.59,3.89,-0.304
4,Bieber,Shane,669456,2019,859,554,0.230,0.236,-0.006,0.393,0.418,-0.025,0.280,0.295,-0.015,3.27,3.80,-0.525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,Weathers,Ryan,677960,2023,217,161,0.318,0.258,0.060,0.563,0.449,0.114,0.401,0.340,0.061,6.89,4.91,1.980
1116,Gonzales,Marco,594835,2023,215,163,0.282,0.286,-0.004,0.431,0.441,-0.010,0.334,0.348,-0.014,5.22,5.18,0.040
1117,Falter,Bailey,663559,2023,214,170,0.317,0.283,0.034,0.515,0.469,0.046,0.369,0.340,0.029,5.21,4.91,0.304
1118,Liberatore,Matthew,669461,2023,208,155,0.282,0.297,-0.015,0.453,0.508,-0.055,0.349,0.383,-0.034,5.71,6.50,-0.785


In [4]:
# Save the concatenated DataFrame as a CSV file in the "resources" folder
pitcher_combined_data.to_csv("Resources/pitcher_combined_data.csv", index=False)

In [5]:
# Filter the DataFrame for the relevant years (2019, 2021, and 2023)
relevant_years = [2019, 2021, 2022]
filtered_data = pitcher_combined_data[pitcher_combined_data['year'].isin(relevant_years)]

# Group the data by player
pitching_data_2023 = pitcher_combined_data[pitcher_combined_data['year'] == 2023]
grouped_data = filtered_data.groupby('player_id')
average_xera = grouped_data['xera'].mean()

# Merge the average wOBA with the 2023 wOBA for each player
xera_2023 = pd.merge(average_xera, pitching_data_2023[['player_id', 'xera']], on='player_id', suffixes=('_avg', '_2023'))

# Merge player names with the result DataFrame
# Merge player names and calculated results
pitcher_output = pd.merge(
    filtered_data[['player_id', 'first_name', 'last_name']].drop_duplicates(),
    xera_2023,
    on='player_id'
)

# Display the result
pitcher_output

,player_id,first_name,last_name,xera_avg,xera_2023
0,458681,Lance,Lynn,3.273333,4.75
1,594835,Marco,Gonzales,4.623333,5.18
2,669456,Shane,Bieber,3.680000,4.91
3,593958,Eduardo,Rodriguez,3.806667,3.49
4,605400,Aaron,Nola,3.440000,3.81
...,...,...,...,...,...
144,665795,Edward,Cabrera,4.050000,4.06
145,676664,JP,Sears,4.490000,4.66
146,678394,Brayan,Bello,3.800000,4.20
147,657376,Clarke,Schmidt,3.510000,4.23


In [6]:
# Calculate z-scores for xera_avg and xera_2023
z_scores_xera_avg = zscore(pitcher_output['xera_avg'])
z_score_xera_2023 = zscore(pitcher_output['xera_2023'])

# Create new columns for z-scores and differences
pitcher_output['z_scores_xera_avg'] = z_scores_xera_avg
pitcher_output['z_score_xera_2023'] = z_score_xera_2023

# Calculate the difference between z-scores
pitcher_output['zscore_difference'] = z_score_xera_2023 - z_scores_xera_avg


# Display the result
pitcher_output

,player_id,first_name,last_name,xera_avg,xera_2023,z_scores_xera_avg,z_score_xera_2023,zscore_difference
0,458681,Lance,Lynn,3.273333,4.75,-1.125002,0.337984,1.462986
1,594835,Marco,Gonzales,4.623333,5.18,0.817354,0.813179,-0.004174
2,669456,Shane,Bieber,3.680000,4.91,-0.539897,0.514801,1.054698
3,593958,Eduardo,Rodriguez,3.806667,3.49,-0.357652,-1.054448,-0.696797
4,605400,Aaron,Nola,3.440000,3.81,-0.885205,-0.700815,0.184390
...,...,...,...,...,...,...,...,...
144,665795,Edward,Cabrera,4.050000,4.06,-0.007548,-0.424538,-0.416990
145,676664,JP,Sears,4.490000,4.66,0.625516,0.238525,-0.386991
146,678394,Brayan,Bello,3.800000,4.20,-0.367243,-0.269824,0.097420
147,657376,Clarke,Schmidt,3.510000,4.23,-0.784490,-0.236670,0.547820


In [7]:
# Sort the DataFrame by greatest increase in z-score difference
sorted_pitcher_output = pitcher_output.sort_values(by='zscore_difference', ascending=True)

# Display the sorted result
sorted_pitcher_output

,player_id,first_name,last_name,xera_avg,xera_2023,z_scores_xera_avg,z_score_xera_2023,zscore_difference
113,669060,Bryse,Wilson,5.575000,4.20,2.186595,-0.269824,-2.456418
60,663567,Peter,Lambert,6.080000,4.95,2.913180,0.559005,-2.354174
123,650633,Michael,King,4.760000,3.27,1.013987,-1.297571,-2.311559
52,621381,Matt,Strahm,4.660000,3.20,0.870109,-1.374929,-2.245038
26,579328,Yusei,Kikuchi,5.410000,4.26,1.949196,-0.203517,-2.152713
...,...,...,...,...,...,...,...,...
121,656629,Michael,Kopech,3.385000,5.64,-0.964338,1.321528,2.285866
107,656731,Tylor,Megill,3.840000,6.38,-0.309692,2.139305,2.448998
100,666201,Alek,Manoah,3.305000,6.18,-1.079441,1.918284,2.997725
22,425794,Adam,Wainwright,4.396667,7.79,0.491230,3.697504,3.206274


In [8]:
# Save the concatenated DataFrame as a CSV file in the "resources" folder
sorted_pitcher_output.to_csv("Resources/sorted_pitcher_output.csv", index=False)

In [9]:
from pybaseball import statcast_batter_expected_stats

# Get data for all qualified batters in 2019, 2021, 2022, 2023
batter_data_2019 = statcast_batter_expected_stats(2019, 150)
batter_data_2021 = statcast_batter_expected_stats(2021, 150)
batter_data_2022 = statcast_batter_expected_stats(2022, 150)
batter_data_2023 = statcast_batter_expected_stats(2023, 150)

# Concatenate the DataFrames
batter_combined_data = pd.concat([batter_data_2019, batter_data_2021, batter_data_2022, batter_data_2023], ignore_index=True)

# Display the column names of the combined DataFrame
batter_combined_data

,last_name,first_name,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff
0,Semien,Marcus,543760,2019,747,556,0.285,0.272,0.013,0.522,0.495,0.027,0.373,0.363,0.010
1,Merrifield,Whit,593160,2019,735,559,0.302,0.280,0.022,0.463,0.431,0.032,0.340,0.324,0.016
2,Acuña Jr.,Ronald,660670,2019,715,439,0.280,0.279,0.001,0.518,0.574,-0.056,0.369,0.390,-0.021
3,Villar,Jonathan,542340,2019,714,472,0.274,0.246,0.028,0.453,0.412,0.041,0.335,0.315,0.020
4,Betts,Mookie,605141,2019,706,505,0.295,0.311,-0.016,0.524,0.577,-0.053,0.380,0.411,-0.031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1289,Schmitt,Casey,669477,2023,218,152,0.205,0.229,-0.024,0.290,0.327,-0.037,0.244,0.273,-0.029
1290,Wong,Kolten,543939,2023,216,150,0.165,0.210,-0.045,0.227,0.284,-0.057,0.216,0.260,-0.044
1291,Madrigal,Nick,663611,2023,212,177,0.267,0.284,-0.017,0.346,0.348,-0.002,0.301,0.311,-0.010
1292,Arroyo,Christian,624414,2023,206,153,0.241,0.231,0.010,0.369,0.341,0.028,0.275,0.269,0.006


In [10]:
# Save the concatenated DataFrame as a CSV file in the "resources" folder
batter_combined_data.to_csv("Resources/batter_combined_data.csv", index=False)

In [11]:
# Filter the DataFrame for the relevant years (2019, 2021, 2022)
relevant_years = [2019, 2021, 2022]
filtered_data = batter_combined_data[batter_combined_data['year'].isin(relevant_years)]

# Filter the 2023 data
batter_data_2023 = batter_combined_data[batter_combined_data['year'] == 2023]

# Group the data by player and calculate the average wOBA for relevant years
grouped_data = filtered_data.groupby('player_id')
average_woba = grouped_data['woba'].mean()

# Merge the average wOBA with the 2023 wOBA for each player
woba_2023 = pd.merge(average_woba, batter_data_2023[['player_id', 'woba']], on='player_id', suffixes=('_avg', '_2023'))

# Merge player names with the result DataFrame
# Merge player names and calculated results
players_off_output = pd.merge(
    filtered_data[['player_id', 'first_name', 'last_name']].drop_duplicates(),
    woba_2023,
    on='player_id'
)

# Display the result
players_off_output

,player_id,first_name,last_name,woba_avg,woba_2023
0,543760,Marcus,Semien,0.352667,0.354
1,593160,Whit,Merrifield,0.313333,0.338
2,660670,Ronald,Acuña Jr.,0.372000,0.424
3,605141,Mookie,Betts,0.372667,0.406
4,645277,Ozzie,Albies,0.331667,0.354
...,...,...,...,...,...
233,670770,TJ,Friedl,0.323000,0.340
234,641584,Jake,Fraley,0.352000,0.349
235,663743,Nick,Fortes,0.308000,0.258
236,608841,Joey,Meneses,0.395000,0.322


In [12]:
z_scores_woba_avg = zscore(average_woba)
z_score_woba_2023 = zscore(woba_2023)


In [13]:
# Calculate z-scores for woba_avg and woba_2023
players_off_output['zscore_woba_avg'] = zscore(players_off_output['woba_avg'])
players_off_output['zscore_woba_2023'] = zscore(players_off_output['woba_2023'])

# Calculate the difference between z-scores
players_off_output['zscore_difference'] = players_off_output['zscore_woba_2023'] - players_off_output['zscore_woba_avg']

# Display the result
players_off_output

,player_id,first_name,last_name,woba_avg,woba_2023,zscore_woba_avg,zscore_woba_2023,zscore_difference
0,543760,Marcus,Semien,0.352667,0.354,0.780199,0.799043,0.018843
1,593160,Whit,Merrifield,0.313333,0.338,-0.484402,0.369942,0.854344
2,660670,Ronald,Acuña Jr.,0.372000,0.424,1.401783,2.676359,1.274576
3,605141,Mookie,Betts,0.372667,0.406,1.423217,2.193621,0.770404
4,645277,Ozzie,Albies,0.331667,0.354,0.105031,0.799043,0.694012
...,...,...,...,...,...,...,...,...
233,670770,TJ,Friedl,0.323000,0.340,-0.173610,0.423579,0.597190
234,641584,Jake,Fraley,0.352000,0.349,0.758765,0.664949,-0.093817
235,663743,Nick,Fortes,0.308000,0.258,-0.655874,-1.775563,-1.119689
236,608841,Joey,Meneses,0.395000,0.322,2.141254,-0.059159,-2.200413


In [14]:
# Sort the DataFrame by greatest increase in z-score difference
sorted_output_batting_woba = players_off_output.sort_values(by='zscore_difference', ascending=False)

# Display the sorted result
sorted_output_batting_woba

,player_id,first_name,last_name,woba_avg,woba_2023,zscore_woba_avg,zscore_woba_2023,zscore_difference
198,672695,Geraldo,Perdomo,0.253000,0.340,-2.424172,0.423579,2.847752
115,656811,Ryan,O'Hearn,0.276000,0.353,-1.684702,0.772224,2.456926
23,641355,Cody,Bellinger,0.311667,0.393,-0.537987,1.844976,2.382963
73,608369,Corey,Seager,0.353333,0.438,0.801633,3.051823,2.250189
177,673490,Ha-Seong,Kim,0.291500,0.360,-1.186363,0.959956,2.146319
...,...,...,...,...,...,...,...,...
168,606992,Eric,Haase,0.319000,0.233,-0.302214,-2.446033,-2.143819
53,516782,Starling,Marte,0.357333,0.278,0.930237,-1.239187,-2.169424
236,608841,Joey,Meneses,0.395000,0.322,2.141254,-0.059159,-2.200413
79,641313,Tim,Anderson,0.343667,0.256,0.490841,-1.829201,-2.320042


In [15]:
# Save the concatenated DataFrame as a CSV file in the "resources" folder
sorted_output_batting_woba.to_csv("Resources/sorted_output_batting_woba.csv", index=False)